In [1]:
import numpy as np
from scipy.optimize import minimize 
from scipy.stats import entropy

In [65]:
# L : number of glycan types
# N : number of lattice points
# P = [p(1,1),...,p(1,N), ... ,p(L,1),...,p(L,N)]  (LN in row-major order)
# W = [[w11,...,w1N], ... ,[wL1,...,wLL]]          (LxL)

def total_entropy(P, L, N):
    return np.sum([entropy(P[i*N:(i+1)*N]) for i in range(L)])

def neg_total_entropy(P, L, N):
    return -np.sum([entropy(P[i*N:(i+1)*N]) for i in range(L)])

def P_neigh(P, N, t, x):  # probability that type t neighbours location x, with bounds checking on x
    if x == 0:
        return P[t][x+1]
    elif x == N-1:
        return P[t][x-1]
    else:
        return P[t][x-1] + P[t][x+1]

def neighbour_constr_1D(P, *args):  # p(t1||t2) = Sum([p(t1,x)*(p(t2,x-1)+p(t2,x+1)) for x from 1 to N])
    L, N, W, t1, t2 = args
    Pt = np.array([P[i*N:(i+1)*N] for i in range(L)]) 
    
    n_prob = np.sum(np.array([P_neigh(Pt, N, t2, i)*Pt[t1][i] for i in range(N)]))
    return n_prob - W[t1][t2]

def Jacobian_1D(P, *args):
    L, N, W, t1, t2 = args
    Pt = np.array([P[i*N:(i+1)*N] for i in range(L)]) 
    jac = np.zeros(L*N)
    
    for x in range(N):
        jac[t1*N + x] = P_neigh(Pt, N, t2, x)
        jac[t2*N + x] = P_neigh(Pt, N, t1, x)
    
    return jac  

def create_constraint_dicts_1D(P, L, N, W):
    return [{'type': 'eq', 'fun': neighbour_constr_1D, 'args': (L, N, W, t1, t2)} 
            for t1 in range(L) for t2 in range(L)]          

In [66]:
# verification

print("Total Entropy Function Tests")
print("[1, 0], [0, 1] =>", total_entropy(P=[1, 0, 0, 1], L=2, N=2))
print("[1, 0, 0, 1] =>", total_entropy(P=[1, 0, 0, 1], L=1, N=4))
print("[1, 1, 1, 1] =>", total_entropy(P=[1, 1, 1, 1], L=1, N=4))
print("scipy entropy [1, 1, 1, 1] =>", entropy([1, 1, 1, 1]))
print("scipy entropy [0.25, 0.25, 0.25, 0.25] =>", entropy([0.25, 0.25, 0.25, 0.25]))

Total Entropy Function Tests
[1, 0], [0, 1] => 0.0
[1, 0, 0, 1] => 0.6931471805599453
[1, 1, 1, 1] => 1.3862943611198906
scipy entropy [1, 1, 1, 1] => 1.3862943611198906
scipy entropy [0.25, 0.25, 0.25, 0.25] => 1.3862943611198906


In [67]:
print("Dictionary Verification Tests")
L = 2
N = 3
P = np.asarray([0.5] * (L*N))
W = np.array([[0.1, 0.9], [0.7, 0.3]])
constr = create_constraint_dicts_1D(P, L, N, W)
print("L = {}\nN = {}\nP = {}\nW = {}\n".format(L, N, P, W))
print("Dictionaries created:", constr, sep="\n")

print("\nJacobian for f(1, 0):")
print(Jacobian_1D(P, L, N, W, 1, 0))

Dictionary Verification Tests
L = 2
N = 3
P = [0.5 0.5 0.5 0.5 0.5 0.5]
W = [[0.1 0.9]
 [0.7 0.3]]

Dictionaries created:
[{'type': 'eq', 'fun': <function neighbour_constr_1D at 0x7f5aaa7efea0>, 'args': (2, 3, array([[0.1, 0.9],
       [0.7, 0.3]]), 0, 0)}, {'type': 'eq', 'fun': <function neighbour_constr_1D at 0x7f5aaa7efea0>, 'args': (2, 3, array([[0.1, 0.9],
       [0.7, 0.3]]), 0, 1)}, {'type': 'eq', 'fun': <function neighbour_constr_1D at 0x7f5aaa7efea0>, 'args': (2, 3, array([[0.1, 0.9],
       [0.7, 0.3]]), 1, 0)}, {'type': 'eq', 'fun': <function neighbour_constr_1D at 0x7f5aaa7efea0>, 'args': (2, 3, array([[0.1, 0.9],
       [0.7, 0.3]]), 1, 1)}]

Jacobian for f(1, 0):
[0.5 1.  0.5 0.5 1.  0.5]


In [68]:
# minimization tests

minimize(neg_total_entropy, P, args=(L, N), constraints=constr)

     fun: -2.197224577336219
     jac: array([0., 0., 0., 0., 0., 0.])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 8
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [69]:
from scipy.optimize import root, fsolve
import math
    
def safelog(x):
    if x <= 0:
        return 0
    return math.log(x)

def lj(x, L, N, W):
    P, Lmd = x[:L*N], x[L*N:]
    # transform to 2D array 
    Pt = np.array([P[i*N:(i+1)*N] for i in range(L)])
    Lmd = np.array([Lmd[i*L:(i+1)*L] for i in range(L)])
    
    dLdp = np.zeros(L*N)
    dLdLmd = np.zeros(L*L)
    
    for t in range(L):
        for i in range(N):
            dLdp[t*N + i] = 1 + safelog(Pt[t][i])  # prevent bounds error 
            dLdp[t*N + i] += np.sum(np.array([(Lmd[t][t1]+Lmd[t1][t])*P_neigh(Pt, N, t1, i) for t1 in range(L)]))
    
    for t1 in range(L):
        for t2 in range(L):
            dLdLmd[t1*L + t2] = np.sum(np.array([Pt[t1][i]*P_neigh(Pt, N, t2, i) for i in range(N)]))
            dLdLmd -= W[t1][t2]
            
    return np.append(dLdp, dLdLmd)

In [70]:
L = 2
N = 3
P = np.asarray([0.7] * (L*N))
Lmd = np.ones(L*L)
W = np.array([[1, 0], [0, 1]])
x = np.append(P, Lmd)
print("L = {}\nN = {}\nP = {}\nLmd = {}\nW = {}\nx = {}\n".format(L, N, P, Lmd, W, x))

lj(x, L, N, W)

L = 2
N = 3
P = [0.7 0.7 0.7 0.7 0.7 0.7]
Lmd = [1. 1. 1. 1.]
W = [[1 0]
 [0 1]]
x = [0.7 0.7 0.7 0.7 0.7 0.7 1.  1.  1.  1. ]



array([ 3.44332506,  6.24332506,  3.44332506,  3.44332506,  6.24332506,
        3.44332506, -0.04      ,  0.96      ,  0.96      ,  0.96      ])

In [71]:
root(lj, x, args=(L, N, W))

    fjac: array([[-3.05071084e-01,  5.64885558e-01,  0.00000000e+00,
         0.00000000e+00, -7.01020582e-01,  0.00000000e+00,
        -2.83616757e-01, -8.93521605e-02, -8.93521651e-02,
         0.00000000e+00],
       [-3.74996687e-01,  1.12982557e-01, -4.09282711e-01,
         5.07918099e-01,  7.87856008e-02,  5.07918099e-01,
         3.59991035e-01,  1.16914029e-01,  1.16914033e-01,
         0.00000000e+00],
       [ 6.66276426e-01,  1.22026508e-01, -7.14214828e-01,
        -1.26147067e-02, -1.56873398e-01, -1.26147067e-02,
        -7.16164861e-02, -2.26493849e-02, -2.26493859e-02,
         0.00000000e+00],
       [-3.75370274e-01,  2.66424225e-02, -3.75370274e-01,
        -4.44234219e-01,  3.94594117e-01,  1.68977678e-01,
        -3.35609815e-01, -2.90253153e-01, -2.90253159e-01,
        -2.33997317e-01],
       [ 1.19993176e-01, -5.13789672e-01,  1.19993176e-01,
         4.46895802e-01, -2.45379401e-01,  2.81807801e-01,
        -3.55322728e-01, -3.02439921e-01, -3.02439934e-01,
 

In [72]:
fsolve(lj, x, args=(L, N, W))

array([ 0.60990493,  0.76299893,  0.60990493,  0.39571324,  0.48069032,
        0.39571324, -1.52199881,  0.65384946,  3.11678358, -3.06908703])